# Week 3

The tasks cover the following topics:
- teleportation
- superdense coding
- phase oracles
- Deutsch, Deutsch-Jozsa and Bernstein-Vazirani algorithms

We recommend to solve the following katas and tutorials before doing these assignments:
- Teleportation kata
- SuperdenseCoding kata
- ExploringDeutschJozsaAlgorithm tutorial 

from https://github.com/Microsoft/QuantumKatas.

## Part I. Phase oracles and algorithms

In this section the oracles are defined as unitaries that introduce phases to certain computational basis states, as described in the lecture.

### Task 1.1. (Phase oracles for Deutsch algorithm)
**Inputs:**
1. a qubit in an arbitrary state (function input)
2. an integer F which defines which function to implement:  
 - F = 0 : $f(x) = 0$
 - F = 1 : $f(x) = 1$
 - F = 2 : $f(x) = x$
 - F = 3 : $f(x) = 1 - x$

**Goal:** implement a unitary that transforms each basis state $|x⟩$ into state $(-1)^{f(x)} |x⟩$.  

You are not allowed to allocate extra qubits.

In [37]:
%kata T11_Test
open Microsoft.Quantum.Math;

operation Task11 (x : Qubit, F : Int) : Unit is Adj+Ctl {
    // ...
    if (F >= 2) {
        Z(x);
    }
    if (F == 1 or F == 3) {
        R(PauliI, 2.0 * PI(), x);
    }
}

Testing F = 0
Testing F = 1
Testing F = 2
Testing F = 3


Success!

### Task 1.2. (Deutsch Algorithm)
**Input:** a quantum operation - the oracle $|x\rangle \rightarrow (-1)^{f(x)}|x\rangle$, where $f(x)$ is one of the functions implemented in task 1.1.

**Output:** $f(0) \oplus f(1)$, i.e., for $f(x) = 0$ or $f(x) = 1$ the output should be $0$, and for $f(x) = x$ or $f(x) = 1 - x$ the output should be $1$.  

You are allowed to call the oracle exactly once.

In [38]:
%kata T12_Test

operation Task12 (oracle : (Qubit => Unit)) : Int {
    // Note: F = 0 and F = 1 are Unbalanced 1-bit functions
    //       F = 2 and F = 3 are Balanced 1-bit functions
    use q = Qubit();
    H(q);
    oracle(q);
    H(q);
    return M(q) == Zero ? 0 | 1;
}

Testing F = 0
Testing F = 1
Testing F = 2
Testing F = 3


Success!

### Task 1.3. (Phase oracle for Deutsch-Jozsa algorithm)
**Input:** two qubits in an arbitrary state (function input)  
**Goal:** implement a unitary that transforms each basis state $|x_1x_2\rangle$ into state $(-1)^{f(x_1, x_2)}$ $|x_1x_2\rangle$, where $f(x_1, x_2) = 1$ if $x_1 = x_2$ and 0 otherwise.

You are not allowed to allocate extra qubits.

In [39]:
%kata T13_Test

operation Task13 (x : Qubit[]) : Unit is Adj+Ctl {
    // ...
    Controlled Z([x[0]], x[1]);
    ApplyToEachCA(X, x);
    Controlled Z([x[0]], x[1]);
    ApplyToEachCA(X, x);
}

Success!

### Task 1.4. (Phase oracle for Bernstein-Vazirani algorithm)
**Inputs:**   
1. N qubits in an arbitrary state (function input)
2. a bit vector of length N represented as `Bool[]` (`true` corresponds to 1 and `false` to 0)  

**Goal:** implement a unitary that transforms each basis state $|x_1x_2...x_n\rangle$ into state $(-1)^{f(x_1, x_2, ..., x_n)} |x_1 x_2...x_n\rangle$, where $f(x_1, x_2, ..., x_n) = \sum_i s_i x_i$ modulo 2 (i.e., the function implemented by the oracle used in Bernstein-Vazirani algorithm).

You are not allowed to allocate extra qubits.

In [40]:
%kata T14_Test

open Microsoft.Quantum.Arrays;

operation Task14 (x : Qubit[], s : Bool[]) : Unit is Adj+Ctl {

    for i in IndexRange(x) {
        if (s[i]) {
            Z(x[i]);
        }
    }
}

Testing on [False,False]
Testing on [True,False]
Testing on [False,True]
Testing on [True,True]
Testing on [False,False,False]
Testing on [True,False,False]
Testing on [False,True,False]
Testing on [True,True,False]
Testing on [False,False,True]
Testing on [True,False,True]
Testing on [False,True,True]
Testing on [True,True,True]


Success!

## Part II. Teleportation and superdense coding

### Task 2.1. Superdense coding using $|\psi^-\rangle = \frac1{\sqrt2}\big(|01\rangle - |10\rangle\big)$

This task considers a modification of the superdense coding protocol in which the pair of qubits shared by Alice and Bob are entangled in a state $|\psi^-\rangle = \frac1{\sqrt2}\big(|01\rangle - |10\rangle\big)$.  

Alice performs the standard message encoding operation, as implemented in SuperdenseCoding kata:  

```
operation EncodeMessageInQubit_Reference (qAlice : Qubit, message : Bool[]) : Unit {
    if (message[0]) {
        Z(qAlice);
    }
    if (message[1]) {
        X(qAlice);
    }
}
```

After performing this operation she sends her qubit to Bob.

Your task is to implement Bob's part of the protocol (the message decoding) to obtain the two bits of Alice's message.

In [41]:
%kata T21_Test

operation Task21 (qBob : Qubit, qAlice : Qubit) : Bool[] {
    // ...
    Z(qBob);
    X(qBob);
    CNOT(qAlice, qBob);
    H(qAlice);
    let Alice = M(qAlice) == One;
    let Bob = M(qBob) == One;
    return [Alice, Bob];
}

Success!

### Task 2.2. Teleportation using $\frac1{\sqrt2}\big(|01\rangle + i|10\rangle\big)$

This task considers a modification of the teleportation protocol in which the pair of qubits shared by Alice and Bob are entangled in a state $\frac1{\sqrt2}\big(|01\rangle + i|10\rangle\big)$, where the first qubit in ket notation denotes Alice's qubit and the second one - Bob's qubit.  

Alice has a qubit in the state $|\psi\rangle = \alpha|0\rangle + \beta|1\rangle$.  
Alice performs the standard message sending operation, as implemented in the Teleportation kata:
```
operation SendMessage_Reference (qAlice : Qubit, qMessage : Qubit) : (Bool, Bool) {
    CNOT(qMessage, qAlice);
    H(qMessage);
    return (M(qMessage) == One, M(qAlice) == One);
}
```
After performing this operation she sends the two measured bits to Bob.

Your task is to implement Bob's part of the protocol (the fix-up), so that he ends up with a qubit in the state $|\psi\rangle$.

You are allowed to introduce a global phase to the teleported state.

In [42]:
%kata T22_Test

operation Task22 (qBob : Qubit, (b1 : Bool, b2 : Bool)) : Unit {
    // ...
    if (not b2) {
        X(qBob);
    }
    if (b1 == b2) {
        Adjoint S(qBob);
    }
    if (b1 != b2) {
        S(qBob);
    }
}

Success!

### Task 2.3. S-gate teleportation

Alice and Bob share a qubit in the state $\frac1{\sqrt2}\big(|00\rangle - i|11\rangle\big)$.  
Alice has a qubit in the state $|\psi\rangle = \alpha|0\rangle + \beta|1\rangle$.  
She wants to send to Bob the state $S|\psi\rangle = \alpha|0\rangle + \beta i|1\rangle$ without Bob applying an $S$ gate to his qubit.  
Alice performs the standard message sending operation, as implemented in the Teleportation kata:
```
operation SendMessage_Reference (qAlice : Qubit, qMessage : Qubit) : (Bool, Bool) {
    CNOT(qMessage, qAlice);
    H(qMessage);
    return (M(qMessage) == One, M(qAlice) == One);
}
```
She sends Bob the return of this operation.

Your task is to implement Bob's part of the protocol (the fix-up), so that he ends up with a qubit in the state $S|\psi\rangle$.  
You can only use Pauli and H gates; you can not use S, T or arbitrary rotation gates.

You are allowed to introduce a global phase to the teleported state.

In [43]:
%kata T23_Test

operation Task23 (qBob : Qubit, (b1 : Bool, b2 : Bool)) : Unit {
    // ...
    
    if (b2) {
        X(qBob);
    }
    if (b1 == b2) {
        Z(qBob);
    }
}

Success!